# Removing bands and fixing dimensions from the Sentinel-2 patches.

The dataset details regarding the bands can be found [in the GEE page](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR).

Two functions are used: `remove_bands` and `fix_image_dimensions`

Just enter the image parent directory to start

In [1]:
import os
import rasterio as rio
import numpy as np

image_directory = r"C:\Users\Kostas\Desktop\GIMA\Module_7\Data\filtered_patches_GEE"


## Removing extra bands

In [2]:
'''
This function reads the GEE Sentinel-2 Image and removes the excess bands. The original images have 23 bands and only the first 12 are needed.
'''

def remove_bands(input_image, output_image):

    # Open the input image
    with rio.open(input_image) as src:
        # Read all the available bands
        bands = src.read()

        # Select bands 1 to 12
        bands_to_keep = bands[0:12]

        # Copy the metadata from the source dataset
        meta = src.meta

        # Update the count of bands to 12
        meta.update(count=12)

    # Write the selected bands to the output image.
    with rio.open(output_image, 'w', **meta) as dst:
        dst.write(bands_to_keep)
    src.close
    dst.close

In [3]:
# Read through the directory and its subfolders and find all the files and remove excess bands.
for root, dirs, files in os.walk(image_directory):
    for file in files:
        if file.endswith('.tif'):
            image = os.path.join(root, file)
            remove_bands(image, image)

In [5]:
# Check if the band count is 12
for root, dirs, files in os.walk(image_directory):
    for file in files:
        if file.endswith('.tif'):
            image = rio.open(os.path.join(root, file))
            assert image.count == 12
            image.close


## Removing excess pixels

In [6]:
'''
This function checks if the images have larger dimensions than the desired 
(they had 135 instead 134 pixels in some cases in either height or width) and cuts off the excess pixels.

I have to use temfile and shutil if I want to replace the OG image. That's because a Permision denied error is thrown, since when cropping
it has to read the input image and it cannot replace it because it has to close it first from the memory. It can't close it though
since it has to read it in the last step. That's why a temp image is created and then it gets moved to replace the original one with shutil.
'''

import rasterio as rio
from rasterio.plot import show
from rasterio.windows import Window
from pathlib import Path
import os
import shutil
import tempfile

def fix_image_dimensions(input_image, output_image, target_size):
    with rio.open(input_image, driver='GTiff') as src:
        height, width = src.height, src.width

        # Clause to check if dimensions are okay
        if height <= target_size and width <= target_size:
            return
        
        # Calculate the coordinates for cropping
        crop_width = min(width, target_size)
        crop_height = min(height, target_size)
        x = (width - crop_width) // 2
        y = (height - crop_height) // 2

        window = Window(x, y, crop_width, crop_height)

        # Create a temporary output file
        with tempfile.NamedTemporaryFile(suffix='.tif', delete=False) as temp_output:
            temp_output_image = temp_output.name

            # Out profile
            profile = src.profile
            profile.update(width=crop_width, height=crop_height)
            
            # Crop excess pixels and save to an output
            with rio.open(temp_output_image, 'w', **profile) as dst:
                cropped_data = src.read(window=window)
                dst.write(cropped_data)
        
        shutil.move(temp_output_image, input_image)
        src.close
        dst.close

In [7]:
# Read through the directory and its subfolders and find all the files and remove excess pixels if they have more.
target_size = 134
for root, dirs, files in os.walk(image_directory):
    for file in files:
        if file.endswith('.tif'):
            image = os.path.join(root, file)
            fix_image_dimensions(image, image, target_size)

In [8]:
# Check if the resulting images have correct dimensions
for root, dirs, files in os.walk(image_directory):
    for file in files:
        if file.endswith('.tif'):
            image = rio.open(os.path.join(root, file))
            assert image.height == 134 and image.width == 134
            image.close

### Depecated part.
Will not go for `.png`, so this is not needed

In [ ]:
# Keeping 3 bands and png output

import os
import rasterio
import numpy as np

# Input directory containing subfolders with TIFF images
input_directory = r"C:\Users\Kostas\Desktop\GIMA\Module_7\Data\filtered_patches_GEE"

# Output directory to save the 3 band PNG images
output_directory = r"C:\Users\Kostas\Desktop\GIMA\Module_7\Data\filtered_patches_GEE_3_bands"

# Iterate over the subfolders and files in the input directory
for root, dirs, files in os.walk(input_directory):
    for file in files:
        if file.endswith('.tif'):
            # Construct the input and output file paths
            input_path = os.path.join(root, file)
            output_path = os.path.join(output_directory, os.path.relpath(root, input_directory), os.path.splitext(file)[0] + '.png')
            
            # Create the output directory if it doesn't exist
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            
            # Open the input TIFF file
            with rasterio.open(input_path) as src:
                # Read the desired bands (RGB, which are 2, 3 and 4 based on https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR)
                bands = src.read([2, 3, 4])
                
                # Copy the metadata from the source dataset
                meta = src.meta
                
                # Normalize the pixel values between 0 and 255 because PNG uses this scale instead of float, like TIFF
                bands = np.interp(bands, (bands.min(), bands.max()), (0, 255)).astype(np.uint8)
                
                # Write the modified bands to the output PNG file
                with rasterio.open(output_path, 'w', driver='PNG', count=3, width=bands.shape[2], height=bands.shape[1], dtype='uint8',) as dst:
                    dst.write(bands)

In [ ]:
im = r"C:\Users\Kostas\Desktop\GIMA\Module_7\Data\filtered_patches_GEE_necessary_bands\11\sentinel2_images_mean_2019-04-01_to_2019-05-01-0000000000-0000000000index_49410_station_110_label_DBL_phase_id_11.tif"
from rasterio.plot import show


In [ ]:
with rasterio.open(im) as src:
    print(src.count)
    show(src)
    print(src.shape)
    

Just some other part. Irrelevant

In [ ]:
import time 
import datetime
def clock(): 
    while True: 
        print(datetime.datetime.now().strftime("%H:%M:%S"), end="\r") 
        time.sleep(1) 

In [ ]:
for i in range(0,10):
    print(i, end='\r')
    time.sleep(0.5)

## Checking about the polygon dimensions

This was used at some point while looking for the irregular dimension problem I had.

In [ ]:
import geopandas as gpd

In [ ]:
dimsWidth = []
dimsHeight = []
for i, polygon in enumerate(modified_gdf['geometry']):
    # Get the minimum and maximum coordinates of the bounding box
    minx, miny, maxx, maxy = polygon.bounds

    # Calculate the dimensions of the polygon
    width = maxx - minx
    height = maxy - miny
    dimsWidth.append(width)
    dimsHeight.append(height)
    # Print the dimensions
    #print(f"Polygon {i+1} - Width: {width}, Height: {height}")

In [ ]:
count = 0
for i in dimsHeight:
    if i != 8000.0:
        count += 1

print(count)

0


In [ ]:
from shapely.geometry import Polygon

# Assuming you have a GeoDataFrame called 'gdf' with polygons in the 'geometry' column

# Create empty lists to store the modified polygons
modified_polygons = []

# Iterate over the polygons in the GeoDataFrame
for i, polygon in enumerate(envelopes_gdf['geometry']):
    # Get the minimum and maximum coordinates of the bounding box
    minx, miny, maxx, maxy = polygon.bounds

    # Calculate the original width and height
    width = maxx - minx
    height = maxy - miny

    # Calculate the desired width (e.g., 8000.0) and adjust the coordinates accordingly
    desired_width = 8000.0
    adjustment = desired_width - width
    adjusted_minx = minx - adjustment / 2
    adjusted_maxx = maxx + adjustment / 2

    # Create a new polygon with the adjusted coordinates
    adjusted_polygon = Polygon([(adjusted_minx, miny), (adjusted_maxx, miny), (adjusted_maxx, maxy), (adjusted_minx, maxy)])

    # Append the adjusted polygon to the list
    modified_polygons.append(adjusted_polygon)

# Create a new GeoDataFrame with the modified polygons
modified_gdf = gpd.GeoDataFrame(envelopes_gdf, geometry=modified_polygons)

# Print the modified GeoDataFrame
print(modified_gdf)

         s_id       lon      lat  alt  alt_dem   gss_id       genus  \
0        5363  13.91670  54.0833    2        0  1050100       Alnus   
1        1554   7.51667  51.7333   60       72  2210500       Salix   
2        3120   8.68333  49.5500  140      261  1050100       Alnus   
3        2021   8.58333  50.0000  100      101  1050100       Alnus   
4        1521   7.83333  51.7000   60       58  1050100       Alnus   
...       ...       ...      ...  ...      ...      ...         ...   
129324   5456  13.75000  50.7333  875      864    10000  perm_grass   
129325    961   7.50000  52.7667   30       33    10000  perm_grass   
129326  20595   8.50000  49.4667   95       86    10000  perm_grass   
129327  19312   8.36667  48.6000  490      622    10000  perm_grass   
129328   8197  10.98330  48.2333  525      504    10000  perm_grass   

                species  phase_id  year  day        date Label  \
0       Alnus glutinosa        60  2017   27  2017-01-27   DBL   
1          Sali

In [ ]:
modified_gdf.to_file(r'C:\Users\Kostas\Desktop\GIMA\Module_7\Data\PEP725\After_2016_sent_from_PEP725\pep725_outputs\PEP725_envelopes_modified.geojson', driver='GeoJSON') 